In [2]:
import paddle
# x = paddle.to_tensor([[-114.6, 10.9, 1.1], [-0.304, 38.07, 69.38], [-0.45, -0.17, 62]])
# tau = paddle.to_tensor([1.55, 1.94, 3.0])
# y = paddle.to_tensor([[-114.6, 10.9, 1.1], [-0.304, 38.07, 69.38], [-0.45, -0.17, 62]])
# result = x.ormqr(tau, y, transpose=True)
# print(result)

x=paddle.empty((2, 3, 5, 7))
print(x.strids)
print(x.shape)

W0926 14:20:52.966668  6597 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 6.1, Driver API Version: 12.0, Runtime API Version: 11.8
W0926 14:20:52.972939  6597 dynamic_loader.cc:314] The third-party dynamic library (libcudnn.so) that Paddle depends on is not configured correctly. (error code is /usr/local/cuda/lib64/libcudnn.so: cannot open shared object file: No such file or directory)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX;


RuntimeError: (PreconditionNotMet) Cannot load cudnn shared library. Cannot invoke method cudnnGetVersion.
  [Hint: cudnn_dso_handle should not be null.] (at /paddle/paddle/phi/backends/dynload/cudnn.cc:64)


In [1]:
import torch
# b = 4
# batch_size = 3
# n = 3
# A = torch.randn(batch_size, n, n)

# # 计算每个批次矩阵的逆以及额外信息
# A_invs, infos = torch.linalg.inv_ex(A, check_errors=True)
# print(infos)
x=torch.empty((2, 3, 5, 7))
print(x.dim_order())
print(x.stride())
print(x.size())
x=torch.empty((2, 3, 5, 7), memory_format=torch.channels_last)
print(x.stride())
print(x.dim_order())
print(x.size())

(0, 1, 2, 3)
(105, 35, 7, 1)
torch.Size([2, 3, 5, 7])
(105, 1, 21, 3)
(0, 2, 3, 1)
torch.Size([2, 3, 5, 7])


In [2]:
import paddle
from paddle import Tensor
import math
import numpy as np

# 设置环境为CPU
paddle.set_device('cpu')
def _len_guards(M: int) -> bool:
    """Handle small or incorrect window lengths."""
    if int(M) != M or M < 0:
        raise ValueError('Window length M must be a non-negative integer')
    return M <= 1

def _extend(M: int, sym: bool) -> bool:
    """Extend window by 1 sample if needed for DFT-even symmetry."""
    if not sym:
        return M + 1, True
    else:
        return M, False

def _truncate(w: Tensor, needed: bool) -> Tensor:
    """Truncate window by 1 sample if needed for DFT-even symmetry."""
    if needed:
        return w[:-1]
    else:
        return w

def _general_cosine(
    M: int, a: float, sym: bool = True, dtype: str = 'float32'
) -> Tensor:
    """Compute a generic weighted sum of cosine terms window.
    This function is consistent with scipy.signal.windows.general_cosine().
    """
    if _len_guards(M):
        return paddle.ones((M,), dtype=dtype)
    M, needs_trunc = _extend(M, sym)
    fac = paddle.linspace(-math.pi, math.pi, M, dtype=dtype)
    w = paddle.zeros((M,), dtype=dtype)
    for k in range(len(a)):
        w += a[k] * paddle.cos(k * fac)
    return _truncate(w, needs_trunc)

def _bartlett(M: int, sym: bool = True, dtype: str = 'float32') -> Tensor:
    """
    Computes the Bartlett window.
    This function is consistent with scipy.signal.windows.bartlett().
    """
    if _len_guards(M):
        A = paddle.ones((M,), dtype=dtype)
        # print(A)
        return A
    M, needs_trunc = _extend(M, sym)
    # print(M)

    n = paddle.arange(0, M, dtype=dtype)
    M = paddle.to_tensor(M, dtype=dtype)
    print(2.0*n/(M-1))
    # print(n)
    w = paddle.where(
        paddle.less_equal(n, (M - 1) / 2.0),
        2.0 * n / (M - 1),
        2.0 - 2.0 * n / (M - 1),
    )
    print(w)

    return _truncate(w, needs_trunc)

window_paddle_bartlett=_bartlett(512)
print(paddle.i0(paddle.to_tensor(14.0, dtype='float32')))
print(np.i0(14.0))
def _kaiser(
    M: int, beta: float, sym: bool = True, dtype: str = 'float32'
) -> Tensor:
    """Compute the Kaiser window.
    This function is consistent with scipy.signal.windows.kaiser().
    """
    if _len_guards(M):
        return paddle.ones((M,), dtype=dtype)
    M, needs_trunc = _extend(M, sym)

    beta = paddle.to_tensor(beta, dtype=dtype)

    n = paddle.arange(0, M, dtype=dtype)
    M = paddle.to_tensor(M, dtype=dtype)
    alpha = paddle.to_tensor((M - 1) / 2.0, dtype=dtype)
    w = paddle.i0(
        beta * paddle.sqrt(1 - ((n - alpha) / alpha) ** 2.0)
    ) / paddle.i0(beta)

    return _truncate(w, needs_trunc)

def _nuttall(M: int, sym: bool = True, dtype: str = 'float32') -> Tensor:
    """Nuttall window.
    This function is consistent with scipy.signal.windows.nuttall().
    """
    a=paddle.to_tensor([0.3635819, 0.4891775, 0.1365995, 0.0106411], dtype=dtype)
    return _general_cosine(
        M, a=a, sym=sym, dtype=dtype
    )


def bartlett(M, sym=True):
    
    # Docstring adapted from NumPy's bartlett function
    if _len_guards(M):
        a= np.ones(M)
        # print(a)
        return a
    M, needs_trunc = _extend(M, sym)
    # print(M)

    n = np.arange(0, M)
    # print(n)
    o =  (M - 1) / 2.0
    print(o)
    w = np.where(np.less_equal(n, o),
                 2.0 * n / (M - 1), 2.0 - 2.0 * n / (M - 1))
    print(w)

    return _truncate(w, needs_trunc)

window_scipy_bartlett = bartlett(512)
# print(window)

np.testing.assert_array_almost_equal(
            window_scipy_bartlett, window_paddle_bartlett.numpy(), decimal=5
        )


Tensor(shape=[512], dtype=float32, place=Place(cpu), stop_gradient=True,
       [0.        , 0.00391389, 0.00782779, 0.01174168, 0.01565558, 0.01956947,
        0.02348337, 0.02739726, 0.03131115, 0.03522505, 0.03913894, 0.04305284,
        0.04696673, 0.05088063, 0.05479452, 0.05870841, 0.06262231, 0.06653620,
        0.07045010, 0.07436399, 0.07827789, 0.08219178, 0.08610567, 0.09001957,
        0.09393346, 0.09784736, 0.10176125, 0.10567515, 0.10958904, 0.11350293,
        0.11741683, 0.12133072, 0.12524462, 0.12915851, 0.13307241, 0.13698630,
        0.14090019, 0.14481409, 0.14872798, 0.15264188, 0.15655577, 0.16046967,
        0.16438356, 0.16829745, 0.17221135, 0.17612524, 0.18003914, 0.18395303,
        0.18786693, 0.19178082, 0.19569471, 0.19960861, 0.20352250, 0.20743640,
        0.21135029, 0.21526419, 0.21917808, 0.22309197, 0.22700587, 0.23091976,
        0.23483366, 0.23874755, 0.24266145, 0.24657534, 0.25048923, 0.25440314,
        0.25831702, 0.26223093, 0.26614481, 0.2

In [4]:
import itertools
import unittest

import numpy as np
from scipy import signal
import math
import paddle
import paddle.audio


class TestAudioFuncitons():
    def test_bartlett_nuttall_kaiser_window(self, n_fft: int=512):
        paddle.disable_static()

        window_scipy_bartlett = signal.windows.bartlett(n_fft)
        window_paddle_bartlett = _bartlett(n_fft)
        np.testing.assert_array_almost_equal(
            window_scipy_bartlett, window_paddle_bartlett.numpy(), decimal=5
        )

        window_scipy_nuttall = signal.windows.nuttall(n_fft)
        window_paddle_nuttall = _nuttall(n_fft)
        np.testing.assert_array_almost_equal(
            window_scipy_nuttall, window_paddle_nuttall.numpy(), decimal=5
        )

        window_scipy_kaiser = signal.windows.kaiser(n_fft, beta=14.0)
        print(window_scipy_kaiser.shape)
        window_paddle_kaiser = _kaiser(n_fft, 14.0)
        print(window_paddle_kaiser.numpy().shape)
        np.testing.assert_array_almost_equal(
            window_scipy_kaiser, window_paddle_kaiser.numpy(), decimal=5
        )


a = TestAudioFuncitons()
a.test_bartlett_nuttall_kaiser_window()

Tensor(shape=[512], dtype=float32, place=Place(cpu), stop_gradient=True,
       [0.        , 0.00391389, 0.00782779, 0.01174168, 0.01565558, 0.01956947,
        0.02348337, 0.02739726, 0.03131115, 0.03522505, 0.03913894, 0.04305284,
        0.04696673, 0.05088063, 0.05479452, 0.05870841, 0.06262231, 0.06653620,
        0.07045010, 0.07436399, 0.07827789, 0.08219178, 0.08610567, 0.09001957,
        0.09393346, 0.09784736, 0.10176125, 0.10567515, 0.10958904, 0.11350293,
        0.11741683, 0.12133072, 0.12524462, 0.12915851, 0.13307241, 0.13698630,
        0.14090019, 0.14481409, 0.14872798, 0.15264188, 0.15655577, 0.16046967,
        0.16438356, 0.16829745, 0.17221135, 0.17612524, 0.18003914, 0.18395303,
        0.18786693, 0.19178082, 0.19569471, 0.19960861, 0.20352250, 0.20743640,
        0.21135029, 0.21526419, 0.21917808, 0.22309197, 0.22700587, 0.23091976,
        0.23483366, 0.23874755, 0.24266145, 0.24657534, 0.25048923, 0.25440314,
        0.25831702, 0.26223093, 0.26614481, 0.2

In [ ]:
# 创建一个随机张量
x = torch.randn(2, 3, 4)

# 创建较小的裁剪边界张量，利用广播机制
min_tensor = torch.tensor([-1.0, -0.5, -0.5, 2])  # 形状为 (2,)
max_tensor = torch.tensor([0.5, 1.0, 0.5, 4])    # 形状为 (2,)

# PyTorch 将自动扩展 min_tensor 和 max_tensor 以匹配 x 的形状
x_clipped = torch.clip(x, min=min_tensor, max=max_tensor)
print("Clipped Tensor with Broadcasting Boundaries:")
print(x_clipped)

# 注意：这里的广播是沿着最后一个维度进行的（对于二维张量来说，就是列方向）
# 因此，min_tensor 和 max_tensor 的每个元素会分别应用到 x 的每一列上

In [ ]:
# 只设置最小值
x_clipped_min = torch.clip(x, min=0.0)

# 只设置最大值
x_clipped_max = torch.clip(x, max=1.0)

print("Clipped Tensor with Only Min Boundary:")
print(x_clipped_min)
print("Clipped Tensor with Only Max Boundary:")
print(x_clipped_max)

In [ ]:
import torch

crow_indices = [0, 2, 4, 0, 2, 3]
col_indices = [0, 1, 0, 1, 2, 4, 3, 0]
values = [1, 2, 3, 4, 5, 6, 7, 8]
sparse_tensor = torch.sparse_csr_tensor(torch.tensor(crow_indices, dtype=torch.int64),
                        torch.tensor(col_indices, dtype=torch.int64),
                        torch.tensor(values), dtype=torch.double)

print(sparse_tensor)


In [ ]:
import paddle

dense_3d_tensor = paddle.to_tensor([[[1, 0, 0], [0, 2, 0], [3, 0, 0]],
                                    [[0, 0, 4], [5, 0, 0], [0, 6, 0]]])

print(dense_3d_tensor.to_sparse_csr())
